## LOOKUP TABLES

Since we don't know much about prospects visiting the public site, we will be creating lookup tables based on RBC client information at the postal code level - i.e., we will be using RBC clients in the prospect's neighbourhood as a proxy for our clients. 

*There are four sources of information we will be referencing:*
- __UCP:__ Client demographics in the month that they opened their first CC
- __Environics:__ PRIZM customer lifestyle segment
- __Transactions:__ CC transactions for specific categories of spending
- __Credit:__ Credit score, balance, and number of other financial institutions

##### Timing 
We want to time how long these programs take to run. We are interested both in real time and CPU time. 

In [ ]:
import time 

start_time = time.time()
start_cpu_time = time.process_time()

#### Set Up

In [ ]:
import os
import numpy as np
import calendar

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.functions import collect_list, regexp_replace, lower
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import year, month, dayofmonth, to_date, trim, concat, col, lit
from functools import reduce

import datetime 
from datetime import datetime as dt
from dateutil.relativedelta import *
import pandas as pd

#### Function to Create Dates

In [ ]:
def month_end_list(num_months): 

    #Define period start and end based on today's date 
    today = datetime.date.today()
    period_start = today.replace(day=1) + relativedelta(months=-num_months)
    period_end = today.replace(day=1)

    #Create list of month ends
    dtrange = pd.date_range(start=period_start, end=period_end, freq='d')
    months = pd.Series(dtrange.month)
    starts, ends = months.ne(months.shift(1)), months.ne(months.shift(-1))
    df = pd.DataFrame({'month_starting_date': dtrange[starts].strftime('%Y-%m-%d'),
                       'month_ending_date': dtrange[ends].strftime('%Y-%m-%d')})
    
    me_list = list(df['month_ending_date'])[:-1]
    return me_list 

In [ ]:
month_end_list(24)

In [ ]:
def month_start_list(num_months): 

    #Define period start and end based on today's date 
    today = datetime.date.today()
    period_start = today.replace(day=1) + relativedelta(months=-num_months)
    period_end = today.replace(day=1)

    #Create list of month ends
    dtrange = pd.date_range(start=period_start, end=period_end, freq='d')
    months = pd.Series(dtrange.month)
    starts, ends = months.ne(months.shift(1)), months.ne(months.shift(-1))
    df = pd.DataFrame({'month_starting_date': dtrange[starts].strftime('%Y-%m-%d'),
                       'month_ending_date': dtrange[ends].strftime('%Y-%m-%d')})
    
    me_list = list(df['month_starting_date'])[:-1]
    return me_list 

In [ ]:
month_start_list(12)

In [ ]:
def get_years_between_dates(start_date_str, end_date_str):
    
    start_date = dt.strptime(start_date_str, '%Y-%m-%d').date()
    end_date = dt.strptime(end_date_str, '%Y-%m-%d').date()

    if start_date > end_date:
        raise ValueError("Start date must be before end date.")

    start_year = start_date.year
    end_year = end_date.year
    return list(range(start_year, end_year + 1))

In [ ]:
get_years_between_dates(month_end_list(24)[0],month_end_list(24)[-1])

#### Function to Check File Paths

In [ ]:
def check_folder(path):

    # Access Hadoop FileSystem
    hadoop_fs = spark._jsc.hadoopConfiguration()
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_fs)
    
    # List subfolders
    files = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path))
    subfolders = [file.getPath().getName() for file in files if file.isDirectory()]
        
    return subfolders

In [ ]:
check_folder("/anaurosevic/cdn0_cards_affinity/lookup")

#### Universal Client Profile

##### Client Base
The UCP base will be comprised of people who opened a credit card for the first time in the last 2 years*. It is important that the credit card is their first product with the bank at the time of purchase - specifically to retain comparability between our incoming prospects. 

**Note: With only 1 year of data, there weren't enough customers at each postal code.*

In [ ]:
month_start_list(24)

In [ ]:
date_filter_string = "open_mth>='" + month_start_list(24)[0] + "'and open_mth<='" +  month_start_list(24)[-1] + "'"
print(date_filter_string)

In [ ]:
open_acct = spark.read.option("basePath","...SALES...").load(
    "...SALES...").filter(date_filter_string)

##### Filter to clients of interest
We want clients who meet the following criteria: 
(1) New clients - not existing 
(2) Opened a credit card 
(3) Opened the credit card online 
(4) Personal clients - not business 

These clients are as close as possible to our prospects and so we can use their client info based on the time they first acquired their CC to impute averages for our true prospects. 

In [ ]:
#(1) Filter to only new clients 
new_clients = open_acct.filter("new_existing == 'NEW'")
new_clients.count()

In [ ]:
#(2) Ensure that they opened a credit card specifically 
new_clients_cc = new_clients.filter("lob == 'Credit Cards'")
new_clients_cc.count()

In [ ]:
#(3) Online channels
new_clients_cc_online = new_clients_cc.filter("channel in ('.com','OLB','Mobile')")
new_clients_cc_online.count()

In [ ]:
#(4) Personal and not business clients 
new_clients_cc_online_personal = new_clients_cc_online.filter("business_ind == 0")

In [ ]:
#Final existing client pool which we will merge onto UCP 
clients_of_interest = new_clients_cc_online_personal.select(
    "open_dt","clnt_no","product_code","student_ind","newcomer_ind")
clients_of_interest.show(10,False)

##### Fix inconsistencies
We want to ensure that we have unique clients who are representative of the cards we are interested in. We will filter out people who: 
- __Have a CC other than our 14 of interest:__ Not informative for predicting CCs
- __Have missing client number:__ Won't be able to link them to UCP
- __Multiple CC sales for a given client:__ We should only observe one account opening for each client especially since we filtered on "new" clients. Investigations into cases where there are multiple CC sales for "new" clients reflect cases where customers purchase a card, close the account, then reopen. These cases are both rare and confusing so we will ignore them for the postal code aggregation.

In [ ]:
credit_cards = ["CLO","IAV","ION","MC4","IOP","GCP","MC1","AVP","MC2","MCP","MV1","PLT","GUS","BAP"]
credit_cards

In [ ]:
clients_cc = clients_of_interest.filter(F.col('product_code').isin(credit_cards))
clients_cc.count()

In [ ]:
#Quick check - any missing client numbers? 
print(clients_cc.filter("clnt_no is null").count())
#Great, we're good to go!
print(clients_cc.count())

In [ ]:
#Keep clients with only one row since people who open then close accounts are messy 
one_row_only = clients_cc.groupBy("clnt_no").count().where("count==1").select("clnt_no")

final_clients = clients_cc.join(
    one_row_only, one_row_only.clnt_no == clients_cc.clnt_no,"inner").drop(
    one_row_only.clnt_no).select(
    "clnt_no","open_dt","product_code","student_ind","newcomer_ind")

final_clients.show(5, False)

In [ ]:
final_clients.count()

##### UCP Merge

In [ ]:
date_filter_string = "MONTH_END_DATE>='" + month_end_list(24)[0] + "'and MONTH_END_DATE<='" + month_end_list(24)[-1] + "'"
print(date_filter_string)

In [ ]:
ucp = spark.read.option(
    "basePath","...CLIENT_PROFILE...").load(
    "...CLIENT_PROFILE...").filter(
    date_filter_string)

In [ ]:
#Merge onto these clients 
client_base_ucp = final_clients.join(ucp,
  (ucp.CLNT_NO == final_clients.clnt_no) & 
  #Snapshot from the month in which they opened their account
  (month(ucp.MONTH_END_DATE) == month(final_clients.open_dt)) &
  (year(ucp.MONTH_END_DATE)== year(final_clients.open_dt)),"inner").drop(
    ucp.CLNT_NO).drop(ucp.MONTH_END_DATE).withColumnRenamed('POST_CD','postal_code')

In [ ]:
client_base_ucp.select('clnt_no','open_dt','product_code','student_ind','newcomer_ind','postal_code').show(5,False)

In [ ]:
#Save
#We will use this client base for credit score/summary as well
cb_ucp = client_base_ucp.select('clnt_no','open_dt','product_code','student_ind','newcomer_ind','postal_code')
cb_ucp.coalesce(1).write.mode("overwrite").parquet("/anaurosevic/cdn0_cards_affinity/lookup/cb_ucp")

##### Formatting Variables

_Variables of interest are:_
- Age
- Generation
- Lifetime Profit Value
- Life Stage Segment
- Sex

In [ ]:
ucp_cols = client_base_ucp.withColumn(
    #Generation
    "gen",
    F.when(
        F.col("GENERATION").isNull(),"unk").when(
        F.col("GENERATION")=="I.Gen Z","z").when(
        F.col("GENERATION")=="F.Gen X","x").when(
        F.col("GENERATION").isin(['G.Early Millennials','H.Late Millennials']), "millennial").otherwise(
        "boomer")).withColumn(
    #Lifetime profit value
    "life_value", 
    F.when(F.col("LIFETM_VAL_5YR_CLNT_SEG_CD").isNull(),"unk").otherwise(lower(F.col("LIFETM_VAL_5YR_CLNT_SEG_CD")))).withColumn(
    #Life stage segment
    "lifestage", 
    F.when(F.col("LOG_COMP_SEG_CD").isNull(),"unk").otherwise(lower(F.col("LOG_COMP_SEG_CD")))).withColumn(
    #Sex
    "sex",
    F.when(F.col("SEX_SEG_CD").isNull(), "unk").when(
    F.col("SEX_SEG_CD")==1,"m").otherwise("f"))

In [ ]:
#Let's grab what's necessary only ~ 
ucp_important = ucp_cols.select(
    "clnt_no","product_code","postal_code",
    "student_ind","newcomer_ind","AGE","gen","life_value","lifestage","sex").withColumnRenamed(
    "AGE","age").withColumnRenamed(
    "product_code","cc")

In [ ]:
ucp_important.show(10,False)

##### Postal Code Aggregation

In [ ]:
ucp_final = ucp_important.drop("clnt_no").filter("postal_code is not null")

In [ ]:
ucp_final.show(5,False) 

In [ ]:
ucp_final_pd = ucp_final.toPandas()

In [ ]:
all_cols = list(ucp_final_pd.columns.values)
cat_cols = [x for x in all_cols if x not in ['postal_code','age','student_ind','newcomer_ind']]
print(cat_cols) 

In [ ]:
dummies = pd.get_dummies(ucp_final_pd[cat_cols], drop_first=False)
dummies.head(5)

In [ ]:
ucp_flags = pd.concat([ucp_final_pd[['postal_code','age','student_ind','newcomer_ind']], dummies], axis=1)

In [ ]:
ucp_flags.columns[ucp_flags.isnull().any()] #Great!

##### Averages

In [ ]:
postal_code_agg = ucp_flags.groupby('postal_code',as_index=False).mean()

In [ ]:
postal_code_agg.head(5)

In [ ]:
postal_code_agg.columns.values

In [ ]:
#Back to spark!
postal_code_agg_spark = spark.createDataFrame(postal_code_agg)

In [ ]:
postal_code_agg_spark.count()

In [ ]:
ucp_lookup_final = postal_code_agg_spark

##### Save

In [ ]:
ucp_lookup_final.coalesce(1).write.mode("overwrite").parquet("/anaurosevic/cdn0_cards_affinity/lookup/ucp")

#### Environics 

In [ ]:
environics_2024 = spark.read.csv(
    "PRIZM_Unique_License_202401_GEO.csv", 
    header=True, 
    mode="DROPMALFORMED"
)
print(environics_2024.count())
environics_2024.printSchema()

In [ ]:
environics_2024 = environics_2024.withColumnRenamed("FSALDU","postal_code").withColumnRenamed("PRIZM","prizm_seg").select(
    "postal_code","prizm_seg")
environics_2024.show(3,False)

In [ ]:
categorical_columns = ["prizm_seg"]

def create_exprs(column_name):
    categories = environics_2024.select(column_name).distinct().\
    rdd.flatMap(lambda x: x).collect()

    return [F.when(F.col(column_name) == category, 1).otherwise(0).\
            alias(f"{column_name}_{category}") for category in categories]

exprs = reduce(lambda acc, col_name: acc + create_exprs(col_name), 
               categorical_columns, [])

result_df = environics_2024.select("postal_code", *exprs)

result_df.printSchema()

##### Save 

In [ ]:
environics_final = result_df
environics_final.coalesce(1).write.mode("overwrite").parquet("/anaurosevic/cdn0_cards_affinity/lookup/environics")

#### Credit Score and Summary

In [ ]:
#Use client base saved earlier 
final_clients = spark.read.load("/anaurosevic/cdn0_cards_affinity/lookup/cb_ucp")

#### (A) Credit Summary

In [ ]:
#The TU table is updated quarterly
#This means that we only have data for the latest Q which ended in Jan
final_clients_dates = final_clients.select(
    'clnt_no','open_dt','postal_code').withColumn(
    'current_month_end',F.last_day('open_dt')).withColumn(
    'next_month_end',F.last_day(F.date_add('current_month_end',1)))

In [ ]:
final_clients_dates.show(5,False)

In [ ]:
years_to_import = get_years_between_dates(month_end_list(24)[0], month_end_list(24)[-1])
print(years_to_import)

In [ ]:
#Create regex for import
regex = "QTR_DT={" + ",".join(f"{year}*" for year in years_to_import) + "}"
print(regex)

base_path = "...TTU_TRADE..."
dynamic_path = base_path+regex
print(dynamic_path)

In [ ]:
tu_all_raw = spark.read.option('basePath', base_path).load(dynamic_path)

In [ ]:
tu_all_raw.select('DT_RPTD').show(5,False)

In [ ]:
#Grab relevant columns and reformat 
tu_all = tu_all_raw.filter(
    #Preferred record
    F.trim(F.col('PREFER_IND'))=='Y').filter(
    #Revolving/open trade types 
    F.trim(F.col("TRD_CATG")).isin(['REV','OPN'])).filter(
    #Credit card specific accounts 
    F.trim(F.col("TRD_SUB_CATG")).isin(["BC","RT","QC"])).withColumn(
    #Last report must be in the last year
    "last_report_date", F.col("DT_RPTD")).withColumn(
    'snapshot_date',F.col('QTR_DT')).withColumn(
    #Convert clnt number to string 
    "clnt_no",F.col("CLNT_NO").cast("string")).withColumnRenamed(
    "BAL","balance").withColumnRenamed(
    "SUB_SH_NAME",'subscriber').select("clnt_no","snapshot_date","last_report_date","balance","subscriber")

In [ ]:
tu_all.show(5,False)

We will grab snapshot dates in the date of card sale AND the following month where they may be reported.
If there are multiple snapshot dates, then we will only grab the first for each client!
Left join is important because even if a client doesn't have a recent credit report, it doesn't mean they're not still in the system.

In [ ]:
clients_tu = final_clients_dates.join(tu_all,
    (final_clients_dates.clnt_no == tu_all.clnt_no) &
     #Snapshot from the month in which they opened their account **OR THE FOLLOWING MONTH**
    ((final_clients_dates.current_month_end == tu_all.snapshot_date) |
    (final_clients_dates.next_month_end == tu_all.snapshot_date)), how="left").drop(
    tu_all.clnt_no)

In [ ]:
#Grab only one row per client: first snapshot date!
min_snapshot = clients_tu.groupBy('clnt_no').agg(F.min('snapshot_date').alias('min_snapshot_date'))

first_tu = clients_tu.join(min_snapshot, 
                           (clients_tu.clnt_no == min_snapshot.clnt_no) &
                           (clients_tu.snapshot_date == min_snapshot.min_snapshot_date),
                           how='inner').drop(min_snapshot.clnt_no).persist()

In [ ]:
#Focus on trades where the last reported date is within the last year 
recent_credit_report = first_tu.filter(
    #Grab all trades for these clients where the last reported date is within the last year! 
    #Assumption: Anything where the last report date is longer than that is likely closed 
    F.col("last_report_date")>=F.add_months(F.col("open_dt"), -12))

In [ ]:
#Interested in total balance across all credit lines and number of unique institutions (OFI = other financial institution) 
summ_credit = recent_credit_report.groupBy(
    'clnt_no').agg(
    F.sum('balance').alias('total_balance'), 
    F.countDistinct('subscriber').alias('num_ofi')).persist()

In [ ]:
#Merge back so that we can get all the people who are still clients (i.e., appear in TU data) but just don't have any recent credit
summ_credit_all_clients = first_tu.select('clnt_no','postal_code').distinct().join(
    summ_credit, ["clnt_no"], how='left').na.fill(0)

In [ ]:
summ_credit_all_clients.show(5,False)

##### Postal Code Aggregation

In [ ]:
#Aggregate at the postal code level 
summ_credit_postal_code = summ_credit_all_clients.groupBy(
    'postal_code').agg(
    F.mean('total_balance').alias('os_credit_balance'),
    F.mean('num_ofi').alias('num_ofi'))

In [ ]:
summ_credit_postal_code.show(5,False)

##### Save

In [ ]:
summ_credit_postal_code.coalesce(1).write.mode("overwrite").parquet(
    "/anaurosevic/cdn0_cards_affinity/lookup/credit_summary")

#### (B) Credit Score

In [ ]:
#Create regex for import
regex = "ME_DT={" + ", ".join(f"{year}*" for year in years_to_import) + "}"
print(regex)

base_path = "...CREDIT_SCORE..."
dynamic_path = base_path+regex
print(dynamic_path)

In [ ]:
#Data is updated quarterly! 
#Most recent quarter on April 29, 2025 is January 31, 2025
cr_vis_raw = spark.read.option("basePath",
                               base_path).load(
    dynamic_path).select(
    F.col('CLNT_NO').cast('string').alias('clnt_no'),
    F.col('CRVIS_RSK_SCOR_VAL').alias('credit_score'),
    F.col('ME_DT').alias('month_end_date'),
    F.col('LST_UPDT_DT_TM').cast('date').alias('last_updated'))

In [ ]:
cr_vis_raw.show(5,False)

In [ ]:
#Merge
cr_vis = final_clients_dates.join(
    cr_vis_raw, 
    (final_clients_dates.clnt_no == cr_vis_raw.clnt_no) & 
    ((final_clients_dates.current_month_end == cr_vis_raw.month_end_date) |
     (final_clients_dates.next_month_end == cr_vis_raw.month_end_date)), 
    how='left').drop(
    cr_vis_raw.clnt_no).drop(
    final_clients_dates.current_month_end).drop(
    final_clients_dates.next_month_end).distinct()

In [ ]:
cr_vis.show(5,False) 

In [ ]:
#Grab only one row per client - first snapshot date
min_me = cr_vis.groupBy('clnt_no').agg(F.min('month_end_date').alias('min_me_dt'))

recent_credit_score = cr_vis.join(min_me, 
                           (cr_vis.clnt_no == min_me.clnt_no) &
                           (cr_vis.month_end_date == min_me.min_me_dt),
                           how='inner').drop(min_me.clnt_no)

In [ ]:
recent_credit_score.show(5,False)

##### Postal Code Aggregation
Some people don't have a credit score record - these are the ones that get dropped.
We will ignore those that get dropped because if we don't have info for a given customer it doesn't necessarily mean that they don't HAVE a credit score or that it's zero. 

In [ ]:
summ_cs_postal_code = recent_credit_score.groupBy(
    'postal_code').agg(
    F.mean('credit_score').alias('credit_score'))

In [ ]:
summ_cs_postal_code.show(5,False)

##### Save

In [ ]:
summ_cs_postal_code.coalesce(1).write.mode("overwrite").parquet("/anaurosevic/cdn0_cards_affinity/lookup/credit_score")

#### Transactions - Lookup
Create a lookup dataset at the postal code level of credit card transactions in specific categories. We are interested in the following categories beause they are key differentiators (points accumulation/rewards) of our CCs: 
- Grocery
- Dining and Food Delivery
- Streaming, Digital Gaming & Subscriptions Purchases
- Rides
- Gas & EV Charging
- Travel > Airlines > Westjet / British Airways
- USD

##### Categories 

In [ ]:
#Create a dictionary for Merchant Category Codes (MCC) to identify relevant categories
mcc_dict =  {
    "westjet": [3180],
    "british_airways": [3150],
    "grocery": [5411],
    "gas_ev": [5541, 5542, 5552],
    "rides": [4121, 4111],
    "games": [5815, 5816, 5817, 5818, 4899],
    "dining": [5812, 5813, 5814]
}

In [ ]:
#Travel subcategories (sourced from data dictionary)
travel_sub_cats = ['Accommodations','Airline','Car Rental','Travel','Travel Agencies']

##### Credit Card Client Base
We want to look at transactions over a full year to account for seasonality so we need to identify clients who had their CCs for the full year (i.e., did not close in the last year and opened before the last year). 

__We need the following filters to identify relevant credit card accounts:__ 
- Open before a year ago
- Not closed anytime in the last year

In [ ]:
month_end_list(24)[-1]

In [ ]:
base_path = "...CLIENT_PROFILE..."
dynamic_path = base_path + "MONTH_END_DATE=" + month_end_list(24)[-1]
print(dynamic_path)

In [ ]:
ucp_recent = spark.read.load(dynamic_path).filter(
    #Credit Card Holders
    F.col('CLNT_PROD_SEG_CD').like('%C%')).withColumnRenamed(
    'CLNT_NO','clnt_no').withColumnRenamed(
    'POST_CD','postal_code').select(
    'clnt_no','postal_code').filter(
    #Null postal codes are not useful because we need to agg at the postal code level
    F.col('postal_code').isNotNull())

#These are all clients who have a credit card at the end of the year 

In [ ]:
ucp_recent.count()

In [ ]:
base_path = "...ACCTS..."
dynamic_path = base_path + "ME_DT=" + month_end_list(24)[-1]
print(dynamic_path)

In [ ]:
#Ownership to identify acct_no 
ownership = spark.read.load(dynamic_path).withColumnRenamed(
    'ACCT_NO','acct_no').withColumnRenamed(
    'CLNT_NO','clnt_no').select(
    'clnt_no','acct_no')

In [ ]:
ownership.show(3,False) 

__We need the following filters:__ 
- Open before a year ago: DT_OPENED > one year prior
- Not closed anytime in the last year: DT_CLS is null
- Credit card: MIF_SRVC_ID == 23

In [ ]:
acct = spark.read.load("...ARRANGEMENT...")

In [ ]:
year_ago = month_end_list(13)[0]
year_ago

In [ ]:
#CC Account to make sure it was open ahead of time 
cc_acct = acct.select('AR_ID','MIF_SRVC_ID','DT_OPENED','DT_CLS').filter(
    F.col('MIF_SRVC_ID')==23).filter(
    F.col('DT_OPENED')<year_ago).filter(
    F.col('DT_CLS').isNull()).withColumn(
    "acct_no", F.col("AR_ID").astype(IntegerType())).select(
    'acct_no').distinct()

In [ ]:
#Merge all customers ~
tx_cb = ucp_recent.join(
    ownership, on='clnt_no', how='inner').join(
    cc_acct, on='acct_no', how='inner').select(
    'clnt_no','postal_code','acct_no').distinct()

In [ ]:
tx_cb.show(5,False)

In [ ]:
#Any nulls at all? 
tx_cb.select(
    [F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in tx_cb.columns]
   ).show()
#No, looks like we're good to proceed :) 

##### Transactions
We want to capture transactions of interest in specific category and calculate at the client level: 
- Total number of purchases in that category in a year
- Among those who purchase, total yearly spend in that category

##### Connect to Teradata

In [ ]:
driver = "com.teradata.jdbc.TeraDriver"
# Define the function to load data from Teradata
def load_data(driver, jdbc_url, sql, user, password):
    return spark.read \
        .format('jdbc') \
        .option('driver', driver) \
        .option('url', jdbc_url) \
        .option('dbtable', '({sql}) as src'.format(sql=sql)) \
        .option('user', user) \
        .option('password', password) \
        .load()
url = "fake_url"
user = "XXXXXXXX"
password = "XXXXXXXX" 

##### (A) Function to Pull Relevant Dates 

In [ ]:
def pull_data(year, month): 

    #Define first and last of month to pull in correct values 
    first, last = calendar.monthrange(year, month)

    fom = datetime.datetime(year, month, 1).strftime('%Y-%m-%d')
    lom = datetime.datetime(year, month, last).strftime('%Y-%m-%d')

    date_range_string = "date"+"'"+fom+"' and date'"+lom+"'"
    
    my_sql = """
        select 
        a.ACCT_NO as acct_no, 
        a.PRCH_TXN_AMT as txn_amt, 
        a.PRCH_TXN_DT_ACTUAL as txn_date, 
        a.PRCH_TXN_CURRENCY as txn_currency,
        b.MRCHNT_CATG_CD as mcc, b.MRCHNT_CATG_ROLLUP_GRP as mcc_cat, b.MRCHNT_SUB_CATG_CD_DESC as mcc_sub_cat
        from
        P3C.CR_CRD_RPTS_PRCH_TXN_DLY AS a
        INNER JOIN 
        P3C.LKUP_MRCHNT_CATG AS b
        ON a.MRCHNT_CATG_CD = b.MRCHNT_CATG_CD
        WHERE a.PRCH_TXN_DT_ACTUAL between """ + date_range_string

    df = load_data(driver, url, my_sql, user, password)

    return df

##### (B) Function to Create Number of Purchases for MCC Categories

In [ ]:
def flag_mcc(df, my_dict):
    
    for i in range(0,len(my_dict)): 
    
        #Define key and value 
        key = list(my_dict)[i]
        value = list(my_dict.values())[i]
        
        temp = df.filter(
            F.col('mcc').isin(value)).groupBy(
            'acct_no').agg(
            #Total spend
            F.sum('txn_amt').alias('spend_' + key),
            F.count(F.lit(1)).alias('num_' + key))
    
        #Save to new dataset 
        if (i==0):
            mcc_stack = temp
        else: 
            mcc_stack = mcc_stack.join(temp, on='acct_no', how='outer')

    return mcc_stack

##### (C) Function to Create Flags for Non-MCC Categories

In [ ]:
def flag_non_mcc(df):
    
    temp_travel = df.filter(F.col('mcc_cat')=='Travel').groupBy(
        'acct_no').agg(
        F.sum('txn_amt').alias('spend_travel'),
        F.count(F.lit(1)).alias('num_travel'))

    temp_airline = df.filter(F.col('mcc_sub_cat')=='Airline').groupBy(
        'acct_no').agg(
        F.sum('txn_amt').alias('spend_airline'),
        F.count(F.lit(1)).alias('num_airline'))

    temp_usd = df.filter(F.col('txn_currency')=='USD').groupBy(
        'acct_no').agg(
        F.sum('txn_amt').alias('spend_usd'),
        F.count(F.lit(1)).alias('num_usd'))

    non_mcc_stack = temp_travel.join(
        temp_airline, on='acct_no', how='outer').join(
        temp_usd, on='acct_no', how='outer')
    
    return non_mcc_stack

##### Complete Function 

In [ ]:
def txn_aggregates(date_range, dictionary): 

    #Create empty dictionary to store everything ~
    final_dictionary = {}

    for i in range(0, len(date_range)):

        date = date_range[i]
        yr = dt.strptime(date, '%Y-%m-%d').year
        mth = dt.strptime(date, '%Y-%m-%d').month
        
        #(1) Pull raw data 
        raw_data = pull_data(yr,mth)

        #(2) MCC flags
        mcc_data = flag_mcc(raw_data, dictionary)

        #(3) Non-MCC flags 
        non_mcc_data = flag_non_mcc(raw_data)

        #(4) Combine all data 
        full_data = mcc_data.join(non_mcc_data, on='acct_no', how='outer')

        #(5) Assign date
        monthyear = datetime.datetime(yr, mth, 1).strftime('%Y_%m')
        final_data = full_data.withColumn('monthyear',F.lit(monthyear))

        #(6) Save file
        folder_path =  "/anaurosevic/cdn0_cards_affinity/lookup/txn/raw/"
        file_path = folder_path + str(date)
        
        final_data.coalesce(
            1).write.mode(
            "overwrite").parquet(
            file_path)

In [ ]:
txn_aggregates(month_end_list(12), mcc_dict)

##### Aggregate at the Account, Client, Postal Code Level 
_Our final goal is to have an average number of purchases and average yearly spend by category at the postal code level. To get there, we need to:_
1. Sum all 12 months of transaction data at the account level to get total purchases and yearly spend for each account
2. Sum all accounts at the client level to get a yearly summary for each client across all accounts
3. Average across all clients in a given postal code to create the final lookup dataset 

In [ ]:
month_end_list(12)

In [ ]:
#Import 
base_path = "/anaurosevic/cdn0_cards_affinity/lookup/txn/raw/"

for i in range (0,len(month_end_list(12))): 

    dynamic_path = base_path + str(month_end_list(12)[i])
    month = spark.read.load(dynamic_path)
    
    if i==0:
        full_12m = month
    else:
        full_12m = full_12m.union(month)

In [ ]:
full_12m.count()

##### (A) Account level yearly

In [ ]:
col_list = list(full_12m.columns)[:-1]
print(col_list)

In [ ]:
#Aggregate at the year level by account
acct_agg = full_12m.groupBy('acct_no').sum().drop('sum(acct_no)','monthyear').toDF(*col_list)

In [ ]:
acct_agg.show(5,False)

In [ ]:
acct_agg.count() 
#We have about 7M clients so this makes sense 

##### (B) Client level yearly 

In [ ]:
client_tx = tx_cb.join(
    acct_agg, on ='acct_no', how='left').drop(
    acct_agg.acct_no)

In [ ]:
client_tx.count()

In [ ]:
#Sum at the client level, ignore account 
#Most clients will only have one account, but some may have multiple, we will deal with that here
col_list = list(client_tx.columns)
col_list.remove('acct_no')
print(col_list)

In [ ]:
client_agg = client_tx.drop(
    'acct_no').groupBy(
    'clnt_no','postal_code').sum().drop(
    'sum(clnt_no)','sum(postal_code)').toDF(*col_list)

In [ ]:
print(tx_cb.select('clnt_no').distinct().count(), client_agg.count()) #Great

##### (C) Postal Code Averages

In [ ]:
col_list = list(client_agg.columns)
col_list.remove('clnt_no')
print(col_list)

In [ ]:
postal_code_agg = client_agg.groupBy('postal_code').avg().drop('avg(clnt_no)').toDF(*col_list)

In [ ]:
postal_code_agg.count() #Great! 

In [ ]:
#Check 
postal_code_agg.select(
    'postal_code','num_westjet','spend_westjet','num_grocery','spend_grocery').show(5,False)

##### (D) Format 
Fill in nulls with zeros and save file.

In [ ]:
txn_postal_code = postal_code_agg.fillna(0)

In [ ]:
#Final check ~ 
txn_postal_code.select(
    'postal_code','num_westjet','spend_westjet','num_grocery','spend_grocery').show(5,False)

In [ ]:
txn_postal_code.count()

##### Save

In [ ]:
txn_postal_code.coalesce(1).write.mode("overwrite").parquet(
    "/anaurosevic/cdn0_cards_affinity/lookup/txn/final/")

#### Final 
Final step is to merge all of this information together to create one source dataset.

##### Reload

In [ ]:
ucp = spark.read.load("/anaurosevic/cdn0_cards_affinity/lookup/ucp")
environics = spark.read.load("/anaurosevic/cdn0_cards_affinity/lookup/environics")
tu_credit_summ = spark.read.load("/anaurosevic/cdn0_cards_affinity/lookup/credit_summary")
tu_credit_score = spark.read.load("/anaurosevic/cdn0_cards_affinity/lookup/credit_score")
txn = spark.read.load("/anaurosevic/cdn0_cards_affinity/lookup/txn/final/")

In [ ]:
#Outer join all of these!
#Environics is the most complete ~ 
print(environics.count())
print(ucp.count())
print(tu_credit_summ.count())
print(tu_credit_score.count())
print(txn.count())

In [ ]:
final_pc_lookup = environics.join(
    ucp, on='postal_code', how='outer').join(
    tu_credit_summ, on='postal_code', how='outer').join(
    tu_credit_score, on='postal_code', how='outer').join(
    txn, on='postal_code', how='outer')

In [ ]:
final_pc_lookup.count()

In [ ]:
final_pc_lookup.printSchema()

In [ ]:
#If environics is null, then we can assign unknown ~ 
#For all other values, we will impute the average :)

final_pc_lookup.filter('prizm_seg_01 is null').count()

In [ ]:
final_pc_lookup.columns

In [ ]:
prizm_cols = [i for i in final_pc_lookup.columns if i.startswith('prizm_seg_')]
print(len(prizm_cols))
prizm_cols.remove('prizm_seg_68')
print(len(prizm_cols))

In [ ]:
final_pc_lookup = final_pc_lookup.fillna(0, subset=prizm_cols)
final_pc_lookup = final_pc_lookup.fillna(1, subset='prizm_seg_68')

In [ ]:
final_pc_lookup.filter('prizm_seg_01 is null').count() #Nice 

##### Save

In [ ]:
final_pc_lookup.coalesce(1).write.mode("overwrite").parquet(
    "/anaurosevic/cdn0_cards_affinity/lookup/postal_code_final/")

#### FSA Lookup
We also need to create an FSA level dataset for postal codes that are not common. Since this model will use postal code information (PII), we want to ensure that postal codes where there are few RBC clients - i.e. <5 - are not being used. For these uncommon postal codes, we will use the FSA-level aggregation instead.

In [ ]:
fsa_lookup = final_pc_lookup.withColumn('FSA', F.substring(F.col('postal_code'),0,3))
fsa_lookup.select('postal_code','FSA').show(5, False) 

In [ ]:
fsa_lookup.columns

In [ ]:
col_list = ['FSA'] + list(fsa_lookup.drop('postal_code','FSA').columns)
print(col_list)

In [ ]:
#Group by FSA
final_fsa_lookup = fsa_lookup.drop('postal_code').groupby('FSA').avg().toDF(*col_list)

In [ ]:
final_fsa_lookup.select('FSA','prizm_seg_07','num_travel','age').show(5, False)

In [ ]:
final_fsa_lookup.count() #Nice

##### Save

In [ ]:
final_fsa_lookup.coalesce(1).write.mode("overwrite").parquet("/anaurosevic/cdn0_cards_affinity/lookup/fsa_final/")

--- END PROGRAM ---

In [ ]:
#Timing summary
end_time = time.time()
end_cpu_time = time.process_time()

real_time_elapsed = end_time - start_time
cpu_time_elapsed = end_cpu_time - start_cpu_time

print(f"Real time: {real_time_elapsed:.2f} seconds")
print(f"CPU time: {cpu_time_elapsed:.2f} seconds")